In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
import os
import ssl
import requests

In [2]:
def get_encode_length(tokenizer, sentence) :
    encoded = tokenizer(sentence, padding=True, truncation=False)

    return len(encoded.input_ids)

In [3]:
def get_encode_data(tokenizer, sentences):
    encoded_inputs = tokenizer(sentences, padding=True, truncation=False)
    input_ids = torch.tensor(encoded_inputs['input_ids'])
    attention_masks = torch.tensor(encoded_inputs['attention_mask'])

    return input_ids, attention_masks

In [62]:
def summarize(input_text, tokenizer, model) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=300,
        min_new_tokens=200,
        no_repeat_ngram_size=2,
        num_beams=5,
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    print(len(input_text), len(response))

    return response

In [32]:
def get_summary(input_file, tokenizer, model) :
    f = open(input_file, mode = 'r')
    return summarize(f.read(), tokenizer, model)

In [47]:
#model_name = 'eenzeenee/t5-base-korean-summarization'
model_name = 'psyche/KoT5-summarization'

In [48]:
os.environ['CURL_CA_BUNDLE'] = '/home/osung/Downloads/kisti_cert.crt'

In [49]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('available device: ', device)
else:
    device = torch.device("cpu")
    print('available device: ', device)

available device:  cuda


In [50]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [51]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [52]:
model = model.to(device)

In [26]:
filepath = '/home/osung/data/korean/summary/news_01.txt'

In [27]:
f = open(filepath, mode='r')

In [28]:
lines = f.read()

In [29]:
lines

'인공지능(AI) 반도체 기업 엔비디아의 주가가 치솟으면서 시가총액 2위인 애플을 위협하고 있다. 7일(현지 시각) 엔비디아 주가는 전 거래일 대비 4.47% 오른 926.69달러를 기록했다. 올해 들어서만 주가가 92% 상승했고, 1년전보다 283% 넘게 급등했다. 엔비디아의 시가총액은 2조3160억 달러로, 순위 2위인 애플(2조6090억 달러)과 격차가 3000억 달러 이내로 좁혀졌다. 엔비디아 주가는 AI 열풍에 승승장구 하고 있다. 시총 2조 달러를 넘어섰고, 구글, 아마존, 사우디 아람코를 잇따라 제치면서 현재 시총 순위 3위까지 올라섰다. 그래픽처리장치(GPU) 개발 기업인 엔비디아는 AI 반도체 시장의 90% 가량을 점유하고 있다. 반면 잇따른 악재에 직면한 애플의 주가는 하락세이다. 애플의 주가는 전날보다 0.07% 하락한 169달러를 기록했다. 애플은 중국 내 아이폰 판매 부진, 반독점 소송, 경쟁사보다 뒤처진 AI 기술 등에 올해 들어 주가가 8.96% 하락했다. 한편 엔비디아 주가가 상승세를 이어가자 임원들은 자사 주식을 팔아 차익을 실현하고 있다. 미국 증권거래위원회(SEC)에 따르면 지난 5일 텐치 콕스 엔비디아 이사는 보유 중이던 엔비디아 주식 20만주를 850.03∼852.50달러에 팔았다. 마크 스티븐스 엔비디아 이사도 지난 4일 852.06∼855.02달러에 1만2000주를 매도했다. 두 이사가 매도한 주식은 21만2000주로 총 1억8000만 달러이다.\n'

In [30]:
response = summarize(lines, tokenizer, model)

730 243


In [31]:
response

'7일(현지 시각) 엔비디아 주가는 전 거래일 대비 4.47% 오른 926.69달러를 기록했다. 올해 들어서만 주가가 92% 상승했고, 1년전보다 283% 넘게 급등했다. 시총 2조 달러를 넘어섰고, 구글, 아마존, 사우디 아람코를 잇따라 제치면서 현재 시 총 순위 3위까지 올라섰다. 그래픽처리장치(GPU) 개발 기업인 엔 비디아는 AI 반도체 시장의 90% 가량을 점유하고 있다. 반면 잇따른 악재에 직면한 애플의 주가가 하락세이다.'

In [33]:
get_summary('/home/osung/data/korean/summary/news_02.txt', tokenizer, model)

4533 330


'GC녹십자는 면역글로불린 IVIG-SN 10%(미국 제품명 알리글로)를 오는 7월 미국 시장에 출시한다. FDA 품목허가로 알리글로는 미국 시장 진입에 성공한 국내 첫 혈액제제가 됐다. 오창공장은 다양한 바이오의약품 생산과 함께 위탁생산(CMO) 물량까지 염두에 두고 설계돼 향후 글로벌 의약품 생산기지로 거듭날 전망이라고 말했다. 이 본부장은 "알리글로의 품질 경쟁력을 바탕으로 고 마진 전략을 수립해 보험사와 처방약급여관리업체(PBM), 전문약국, 유통사까지 아우르는 수직 통합 채널 계약을 맺을 것이라며 "이를 통해 미국 사보험 가입자의 약 75%에 알리글라로를 등재시키겠다고 밝혔다.'

In [34]:
get_summary('/home/osung/data/korean/summary/news_03.txt', tokenizer, model)

764 278


'8일 오전 9시10분 기준 코스피 지수는 전일 대비 31.41(1.19%) 오른 2679.03을 나타냈다. 시가총액 상위 종목에서는 SK하이닉스가 4%대, 삼성전자가 1%대로 반도체주가 강세다. 이날 코스닥 지수는 지난 날 대비 5.71(0.66%) 상승한 869.08을 기록했다. 투자자별로는 개인이 456억원 순매도고 외국인과 기관이 384억원, 118억원순매수다. 업종별로는 반도체가 2%대 강세, 종이목재, 방송서비스, 오락문화가 1%대 약세이다. 에코프로비엠, 에코프로는 약보합을 보인다.'

In [35]:
get_summary('/home/osung/data/korean/summary/news_04.txt', tokenizer, model)

989 323


"포스코 포항제철소(소장 천시열)는 5기 코크스 오븐 고온밀폐공간 무인설비 점검을 위한 '이동로봇 통합관제시스템'을 시범 적용하는 데 성공했다고 7일 밝혔다. 포스코는 사람이 직접 점검하기 어려운 고위험개소의 설비를 안전하게 관리하고자 이동로봇을 적극 도입하고 있다. 하지만 운영 환경에 따라 다양한 종류의 로봇이 필요한 만큼 증가하는 로봇을 유기적으로 관리하는 데 어려움이 있었다 포스코홀딩스 미래기술연구원 친환경제조공정솔루션센터는 이러한 문제를 해결하고자 다양한 로봇의 효율적 관리가 가능한 포스코형 로봇편대 자율운전 통합 관제 플랫폼(이하 PosRX) 프로토타입을 개발했다."

In [53]:
get_summary('/home/osung/data/korean/summary/news_10.txt', tokenizer, model)

1607 307


'패션 스토리 회사로 진화하는 F&F 24일 업계에 따르면 최근 들어 콘텐츠 기업에 대한 패션기업들의 투자가 잇따르고 있다. 글로벌 무대에서 통하는 K스토리와 패션 브랜드를 결합하기 위한 시도로 해석된다. 왁티는 국제축구연맹(FIFA)·국제올림픽위원회(IOC) 등과 협력해 글로벌 국제대회의 역사와 관련된 라이선스 상품을 판매하고 있으며 브랜드를 내놓기 전에 차별화된 스토리부터 탄탄히 만들겠다는 것이 한섬의 전략으로 풀이된다. LF와 삼성물산 등도 자체 콘텐츠 개발에 적극적으로 투자하고 있으며 라방(라이브 방송)과 유튜브 콘텐츠를 만들기 위해서다.'

In [58]:
get_summary('/home/osung/data/korean/summary/news_10.txt', tokenizer, model)

1570 415


'24일 업계에 따르면 최근 들어 콘텐츠 기업에 대한 패션기업들의 투자가 잇따르고 있다. 글로벌 무대에서 통하는 K스토리와 패션 브랜드를 결합하기 위한 시도로 해석된다. 한섬이 얼마 전 스포츠 콘텐츠라는 새로운 영역을 개척한 왁티라는 스타트업에 투자한 것도 비슷한 이유에서다. 콘텐츠 대어()들이 넷플릭스 같은 글로벌 OTT들의 구애를 받고 있는 가운데 소규모 제작사에는 패션회사들이 구애가 이어지고 있는 것이다. F&F의 김창수 회장은 최근 드라마 제작사인 빅토리콘텐츠에 235억원을 투자했다. 이미 여섯 번째 콘텐츠기업 투자이다. 브랜드에 내놓기 전에 차별화된 스토리부터 탄탄히 만들겠다는 것이한섬의 전략으로 풀이된다. LF와 삼성물산 등도 자체 콘텐츠 개발에 적극적으로 투자하고 있어 콘텐츠 협력으로 자사몰을 강화하려는 전략이라고 말했다.'

In [59]:
model_name = 'eenzeenee/t5-base-korean-summarization'

In [60]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

In [38]:
get_summary('/home/osung/data/korean/summary/news_01.txt', tokenizer, model)

Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 128). Running this sequence through the model will result in indexing errors


730 169


'엔비디아의 주가가 치솟으면서 시가총액 2위인 애플을 위협하고 있으며 엔비디아 주가는 AI 열풍에 승승장구하며 시총 순위 3위까지 올라섰다. 그래픽처리장치 개발 기업인 엔비디아는 중국 내 아이폰 판매 부진과 반독점 소송 등으로 하락세를 보이고 있는 반면 임원들은 자사 주식 팔아 차익 실현을하는 모습이다.'

In [39]:
get_summary('/home/osung/data/korean/summary/news_02.txt', tokenizer, model)

4533 170


'GC녹십자는 미국 시장 진입에 성공한 국내 첫 혈액제제인 알리글로의 생산 및 출하 준비로 활기를 띠고 있으며 2028년에는 3억 달러 매출을 달성하는 것이 목표라고 말했다. GG녹십자 글로벌 사업본부장은 전문약국 채널과 계약해 판매가도 높이고 면역글로불린 제조공정에서 자체적인 기술력도 확보했다고 밝혔다.'

In [40]:
get_summary('/home/osung/data/korean/summary/news_03.txt', tokenizer, model)

764 180


'반도체 훈풍 덕분에 국내 증시가 상승 출발했다. 코스피 지수는 2679.03을 나타냈고 투자자별로는 개인 2925억원 순매도고 외국인과 기관이 각각 209억 원과 2760억원은 매수다..코스닥 지수도 전일 대비 5.71 오른 869 를 기록했고 시가총액 상위 종목에서는 SK하이닉스가 4 삼성전자 1로 반도체주가 강세이다.'

In [41]:
get_summary('/home/osung/data/korean/summary/news_04.txt', tokenizer, model)

989 157


'포스코 포항제철소는 고위험개소의 설비를 안전하게 관리하고자 이동로봇을 적극 도입하고 있으며 PosRX 플랫폼 기술로 고온 밀폐 공간 무인설비 점검 관제 시스템 개발에 성공했다. 또한 향후 그룹사로 확대 적용할 계획이며 로봇 전환 및 제어 등 다양한 분야에서 활용될 것으로 기대된다.'

In [61]:
get_summary('/home/osung/data/korean/summary/news_10.txt', tokenizer, model)

Token indices sequence length is longer than the specified maximum sequence length for this model (1115 > 128). Running this sequence through the model will result in indexing errors


1570 194


'글로벌 무대에서 통하는 K스토리와 패션 브랜드를 결합하기 위해 콘텐츠 기업에 대한 투자가 잇따르고 있는 가운데 F앤F의 김 회장은 벌써 여섯 번째로 빅토리콘텐츠에235억 원을 투입했다. MLB 같은 OTT들과 달리 소규모 제작사에는 옷 잘 입는 일반인 인플루언서가 소비자와 직접 소통하는 등 차별화된 스토리부터 탄탄히 만들겠다는 전략으로 풀이된다.'

In [63]:
get_summary('/home/osung/data/korean/summary/news_10.txt', tokenizer, model)

1570 282


'글로벌 무대에서 통하는 K스토리와 패션 브랜드를 결합하기 위해 콘텐츠 기업에 대한 투자가 잇따르고 있는 가운데 F앤F의 김 회장은 벌써 여섯 번째로 빅토리콘텐츠에235억 원을 투입했다. MLB 같은 OTT들과 달리 소규모 제작사에는 옷 잘 입는 일반인 인플루언서가 소비자와 직접 소통하는 등 차별화된 스토리부터 탄탄히 만들겠다는 전략으로 풀이된다.LF는 PD 및 미디어 커머스 팀 등이 관련 프로그램 개발 중 알 매  경기 정보 1.1 조 초 박 사회 참여 역사 커뮤니티 빅 규모의1박2 연 장 맞매조에1'